# CNTL temperature dynamic system

## Import package

In [1]:
import numpy as np
import joblib as jl

from matplotlib import pyplot as plt

## Load data

In [2]:
case  = 'NCRF'
fname = f'/home/b11209013/2024_Research/MPAS/LRF/Tseries_file/{case}.joblib'

data = jl.load(fname)
tpc1 = data['pc1']['t']
tpc2 = data['pc2']['t'] # unit in K

# time 
time = np.linspace(0, 32, 33)*0.25 # unit in day

## Compute finite different of the PCs

### Functions

In [3]:
def finite_difference(t_series, time):
    t_derv = np.empty_like(t_series)

    t_derv[0]    = (t_series[1]  - t_series[0])   / (time[1]      - time[0])
    t_derv[-1]   = (t_series[-1] - t_series[-2])  / (time[-1]     - time[-2])
    t_derv[1:-1] = (t_series[2:] - t_series[:-2]) / (t_series[2:] - t_series[:-2])

    return t_derv

### Compute the derivartive of the PC time series

In [4]:
tpc1_dt = np.gradient(tpc1, time, edge_order=2) # unit in K/day
tpc2_dt = np.gradient(tpc2, time, edge_order=2)

## Constructing dynamic system

In [5]:
t_tend = np.stack([tpc1_dt, tpc2_dt], axis=0)
t_vec  = np.stack([tpc1, tpc2], axis=0)

M  = (t_tend @ t_vec.T) @ np.linalg.inv(t_vec @ t_vec.T) # unit in 1/day

print(M)

[[-0.29750463 -1.41484596]
 [ 0.74823703  0.33479996]]


## Computing e-folding time scale

In [6]:
eigvals, eigvec = np.linalg.eig(M)
print('Eigenvalue of the LIM: \n', eigvals)
print('Eigenvectors of the LIM: \n', eigvec)

# growth/decay time
gd_time = 1/np.real(eigvals)
period  = 2*np.pi / np.imag(eigvals)
print('\n')
print('Growth/Decay time of CCKW: ', gd_time[0])
print('Period of CCKW: ', period[0])

Eigenvalue of the LIM: 
 [0.01864766+0.97912607j 0.01864766-0.97912607j]
Eigenvectors of the LIM: 
 [[ 0.80875688+0.j          0.80875688-0.j        ]
 [-0.18071956-0.55968987j -0.18071956+0.55968987j]]


Growth/Decay time of CCKW:  53.62602523034597
Period of CCKW:  6.417136148630577
